In [ ]:
!pip install cloud-tpu-client==0.10 torch==1.13.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.13-cp38-cp38-linux_x86_64.whl tensorboard-plugin-profile

In [ ]:
#1.8.0

In [ ]:
!pip install google-api-python-client==1.12.1

In [ ]:
!pip install pytorch-lightning bert_score datasets rouge_score evaluate transformers wandb torchmetrics

In [1]:
!git clone https://github.com/Myashka/CQA_RLHF.git new_clone

Cloning into 'new_clone'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 194 (delta 127), reused 128 (delta 64), pack-reused 0
Receiving objects: 100% (194/194), 59.27 KiB | 2.47 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [ ]:
!pip uninstall torchvision

In [14]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import yaml

In [7]:
trainer_config = dict(
    seed=42,
    model_name="EleutherAI/gpt-neo-125M",
    wandb=dict(
        api="",
        project_name="CQA_RLHF",
        args=dict(job_type="train", group="sft", name="125M-freezed"),
    ),
    data=dict(
        data_dir=r"/content/drive/MyDrive/Diploma/data/1.0-data-div-ans-sep.json",
        max_length=256,
        batch_size=2,
        pad_for_tpu = True
    ),
    model_params=dict(
        learning_rate=2e-5,
        do_freeze=True,
        use_cache=False,
        warmup_steps=100,
        adam_betas=[0.9, 0.95],
        weight_decay=0.001,
        do_compute_metrics = True,
        do_compute_bertscore = False
    ),
    trainer=dict(
        checkpoint=dict(
            every_n_train_steps=100,
            dirpath=r"/content/drive/MyDrive/Diploma/Checkpoints",
        ),
        ckpt_path=None,
        params=dict(
            accelerator="tpu",
            max_epochs=3,
            auto_scale_batch_size=False,
            accumulate_grad_batches=1,
            gradient_clip_val=1,
            precision="16",
            val_check_interval=50,
            overfit_batches=0,  # 0 for train
            num_sanity_val_steps=2,
            log_every_n_steps = 1,
            limit_train_batches =100
        ),
    ),
)

with open("trainer_config.yaml", "w") as outfile:
    yaml.dump(trainer_config, outfile, default_flow_style=False)

In [9]:
!python /content/new_clone/sft/lightning/pl_train.py --config_file /content/trainer_config.yaml

2023-02-22 22:53:18.571922: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 22:53:19.794301: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-22 22:53:19.794418: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-22 22:53:19.794437: W tensorfl